# GEE 教程（六）：不限大小下载哨兵数据

In [1]:
import os
import ee
import geemap
ee.Initialize()
geemap.set_proxy(port="7890")

## 1 数据准备

### 1.1 加载数据

In [2]:
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
huainan = geemap.geojson_to_ee("./input/huainan.json")

### 1.2 数据预处理

In [3]:
# 将ee.FeatureCollection转换为ee.geometry格式的最小外接矩形
roi = huainan.geometry().bounds()

In [4]:
# 借助 QA60 字段去云
def maskS2clouds(image):
    qa = image.select('QA60')
    # Bits 10 and 11是云，我们要把它mask掉
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 1
    # 这两个标志都应该设置为0，表示条件明确。
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
        .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    # 哨兵的像元值是反射率的10000倍，要除以10000
    return image.updateMask(mask).divide(10000)

In [5]:
# 筛选出待下载的哨兵影像数据
collection = s2.filterDate('2022-01-01', '2022-12-31') \
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
              .filterBounds(huainan) \
              .map(maskS2clouds) \
              .select('B[1-8]')

In [6]:
img = collection.first().clip(roi)

In [7]:
composite = collection.median().clip(roi)

### 1.3 数据显示

In [8]:
rgbVis = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
}

In [9]:
# 设置显示样式：color代表边界颜色；fillcolor代表填充颜色
styling = {
    'color': 'red',
    'fillColor': '00000000'
}

In [10]:
Map = geemap.Map()
# Map.addLayer(img, rgbVis, '淮南市Sentinel-2影像')
Map.addLayer(composite, rgbVis, '淮南市S2影像')
Map.addLayer(huainan.style(**styling), {}, '淮南市边界')
Map.centerObject(huainan, 9)
Map

Map(center=[32.469746984879926, 116.77362718431694], controls=(WidgetControl(options=['position', 'transparent…

## 2 下载影像

### 2.1 配置输出目录

In [11]:
work_dir = os.path.join(
    os.path.expanduser("E:\\researchData\\geeDownloads"),
    'tif'
)
if not os.path.exists(work_dir):
    os.makedirs(work_dir)

In [12]:
out_tif = os.path.join(work_dir, "S2_SR_2022_huainan.tif")

### 2.2 下载数据

In [13]:
# 下载哨兵影像数据
geemap.download_ee_image(
    image=composite,
    filename=out_tif,
    region=roi,
    crs="EPSG:4326",
    scale=10,
)

Consider adjusting `region`, `scale` and/or `dtype` to reduce the S2_SR_2022_huainan.tif download size (raw: 3.71 GB).


S2_SR_2022_huainan.tif: |          | 0.00/3.71G (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None
